In [1]:
from models import ConvLSTM
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

rnn_input_size:  1728
hidden_dim:  64
kernel_size:  3
Using device: cuda


In [2]:
batch_size = 8
inputs_shape = (batch_size, 1, 45, 6, 6, 6)
labels_shape = (batch_size,)
inputs = torch.randn(inputs_shape).to(device)
labels = torch.randint(low=0, high=26, size=labels_shape).to(device)

cnn_flattened_size = inputs_shape[1] * inputs_shape[2] * inputs_shape[3] * inputs_shape[4]

print("inputs shape", inputs.shape)
print("labels shape", labels.shape)

inputs shape torch.Size([8, 1, 45, 6, 6, 6])
labels shape torch.Size([8])


In [3]:
model = ConvLSTM(45, 64, 3, 2)

model = model.to(device)

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

model.train()

rnn_input_size:  1728
hidden_dim:  64
kernel_size:  3


ConvLSTM(
  (conv3d_lstms): ModuleList(
    (0): Conv3d(45, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
    (1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  )
  (rnn_cells): ModuleList(
    (0): LSTMCell(1728, 64)
    (1): LSTMCell(1728, 64)
  )
  (attention): ChannelAttention(
    (fc1): Linear(in_features=45, out_features=45, bias=True)
    (fc2): Linear(in_features=45, out_features=45, bias=True)
  )
  (fc): Linear(in_features=64, out_features=3, bias=True)
)

In [4]:
for epoch in range(1):
    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

    # print statistics
    print('[%d] loss: %.3f' % (epoch + 1, loss.item()))

print('Finished Training')
print(outputs.shape)
print(outputs.cpu().detach().numpy())
print(outputs.cpu().detach().numpy().sum(axis=1))

torch.Size([8, 45])
Shape of x after fc1:  torch.Size([8, 45])
Shape of attention_weights:  torch.Size([8, 45])
Shape of x_t after attention:  torch.Size([8, 45, 6, 6, 6])
Shape of x_t after conv3d_lstm:  torch.Size([8, 64, 6, 6, 6])
Shape of x_t after view:  torch.Size([8, 13824])


RuntimeError: input has inconsistent input_size: got 13824 expected 1728